In [3]:
print("""
@File         : Calculating a trailing stop order price.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2024-12-31 22:22:34
@Email        : cuixuanstephen@gmail.com
@Description  : 计算追踪止损订单价格
""")


@File         : Calculating a trailing stop order price.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2024-12-31 22:22:34
@Email        : cuixuanstephen@gmail.com
@Description  : 计算追踪止损订单价格



In [4]:
import pandas as pd
import numpy as np

In [7]:
%cd ../

d:\Data-Analysis-and-Science\PDC3rd


> In the pandas 2.2 series, there is a bug that prevents the preceding code block from running, instead throwing a `ValueError`. If affected by this bug, you can alternatively run `pd.read_csv` without the `dtype_backend` argument, and add in a call to `pd.DataFrame.convert_dtypes` instead.

In [9]:
if not pd.__version__.startswith('2.2'):
    df = pd.read_csv('DATA/NVDA.csv', usecols=['Date', 'Close'],
                     parse_dates=['Date'], index_col=['Date'], 
                     dtype_backend='numpy_nullable',)
else:
    df = pd.read_csv('DATA/NVDA.csv', usecols=['Date', 'Close'],
                     parse_dates=['Date'], index_col=['Date'],).convert_dtypes(dtype_backend='numpy_nullable')

df    

,Close
Date,
2020-01-02,59.977501
2020-01-03,59.017502
2020-01-06,59.264999
2020-01-07,59.982498
2020-01-08,60.095001
...,...
2023-12-22,488.299988
2023-12-26,492.790009
2023-12-27,494.170013


In [12]:
ser = df.squeeze()
ser.head()

Date
2020-01-02    59.977501
2020-01-03    59.017502
2020-01-06    59.264999
2020-01-07    59.982498
2020-01-08    60.095001
Name: Close, dtype: Float64

In [11]:
ser_cummax = ser.cummax()
ser_cummax.head()

Date
2020-01-02    59.977501
2020-01-03    59.977501
2020-01-06    59.977501
2020-01-07    59.982498
2020-01-08    60.095001
Name: Close, dtype: Float64

In [13]:
ser.cummax().mul(.9).head()

Date
2020-01-02    53.979751
2020-01-03    53.979751
2020-01-06    53.979751
2020-01-07    53.984248
2020-01-08    54.085501
Name: Close, dtype: Float64

In [14]:
ser.cummin().mul(1.1).head()

Date
2020-01-02    65.975251
2020-01-03    64.919252
2020-01-06    64.919252
2020-01-07    64.919252
2020-01-08    64.919252
Name: Close, dtype: Float64

In [15]:
stop_prices = ser.cummax().mul(0.9)
ser[ser <= stop_prices]

Date
2020-02-24         68.32
2020-02-25     65.512497
2020-02-26     66.912498
2020-02-27     63.150002
2020-02-28     67.517502
                 ...    
2023-10-27         405.0
2023-10-30    411.609985
2023-10-31    407.799988
2023-11-01        423.25
2023-11-02    435.059998
Name: Close, Length: 495, dtype: Float64

如果我们只关心确定当前值低于累积最大值的第一天，我们可以使用 `pd.Series.idxmax` 方法。此方法的工作原理是首先计算 `pd.Series` 中的最大值，然后返回遇到该最大值的**第一行索引**：

In [16]:
(ser  <= stop_prices).idxmax()

Timestamp('2020-02-24 00:00:00')